## Lab 5: Recap on Dimensionality Reduction Techniques
You are required to use your implementations from previous labs, but for the new algorithms introduced today you can use external libraries.

In [ ]:
import numpy as np
import pandas as pd
import gzip
import struct
import sklearn
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

from IPython.display import Image


# Exercise 1
Download the ```fashion mnist``` dataset available at the following [link](https://github.com/zalandoresearch/fashion-mnist#get-the-data).

**Today we are going to see how to program an autoencoder, next time (November 10th) we will see the solutions to the whole exercise.**

In [ ]:
def read_idx(filename):
    with gzip.open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

In [ ]:
Xtrain = read_idx('../Datasets/fashion_mnist//train-images-idx3-ubyte.gz')
ytrain=read_idx('../Datasets/fashion_mnist/train-labels-idx1-ubyte.gz')

Xtest = read_idx('../Datasets/fashion_mnist//t10k-images-idx3-ubyte.gz')
ytest =read_idx('../Datasets/fashion_mnist/t10k-labels-idx1-ubyte.gz')

In [ ]:
print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

In [ ]:
labels_dict = { 0: 'T-shirt/top',
                1: 'Trouser',
                2: 'Pullover', 
                3: 'Dress', 
                4: 'Coat', 
                5: 'Sandal', 
                6: 'Shirt', 
                7: 'Sneaker',
                8: 'Bag', 
                9: 'Ankle boot' }

In [ ]:
image = Xtrain[0] #first image in the dataset
label = ytrain[0]

print(f"Class: {label} ({labels_dict[label]})")
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
n_row = 4
n_col = 5

n = n_row * n_col
images = Xtrain[:n]
lab = ytrain[:n]

fig, ax = plt.subplots(n_row, n_col, figsize = (1.5*n_col, 2*n_row))
for i in range(n):
    ax_ = ax[i//n_col, i%n_col]
    ax_.imshow(images[i], cmap='gray')
    ax_.set_title(labels_dict[lab[i]])
plt.tight_layout()
plt.show()

In [ ]:
#normalize it so that the values are in the [0,1] range
Xtrain = Xtrain/255
Xtest = Xtest/255

In [ ]:
Xtrain = Xtrain - Xtrain.mean(axis=0)
Xtest = Xtest - Xtest.mean(axis=0)

In [ ]:
# reshape the images
# this is the format in which you need your data for today

Xtrain = Xtrain.reshape((Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]))
Xtest = Xtest.reshape((Xtest.shape[0], Xtest.shape[1]*Xtest.shape[2]))

print(Xtrain.shape)
print(Xtest.shape)

The above preprocessing steps are required for all the algorithms you will be working with during this lab.

The following applies only to the autoencoder. 

In [ ]:
# shaping necessary for the autoencoder

X_train = Xtrain.reshape((Xtrain.shape[0], 1, 28, 28))
X_test = Xtest.reshape((Xtest.shape[0], 1, 28, 28))

In [ ]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
train_set = torch.utils.data.TensorDataset(X_train, X_train)
train_set, val_set = torch.utils.data.random_split(train_set, [round(len(train_set)*0.8), round(len(train_set)*0.2)])
test_set = torch.utils.data.TensorDataset(X_test, X_test)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 1, shuffle = False)

In [ ]:
# Define the model class
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # define the encoder
        self.encoder = nn.Sequential(
            nn.Linear(in_features=784, out_features=1000),
            nn.ReLU(),
            nn.Linear(1000, 250),
            nn.ReLU(),
            nn.Linear(250, 125),
            nn.ReLU(),
            nn.Linear(125, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  
        )
        # the output of the encoder is the projection in the bottleneck (in this case, a 2-dimensional space)

        # define the decoder
        self.decoder = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 125),
            nn.ReLU(),
            nn.Linear(125, 250),
            nn.ReLU(),
            nn.Linear(250, 1000),
            nn.ReLU(),
            nn.Linear(1000, 784),
            nn.Sigmoid()
        )
        # the output of the decoder is the reconstructed image which was passed as input

    def forward(self, x):
        encoded= self.encoder(x.flatten(1)) 
        decoded = self.decoder(encoded)
        return decoded.view(-1,1,28,28), encoded

In [ ]:
model = Autoencoder()

In [ ]:
model

In [ ]:
def train(model, train_loader, val_loader, loss_func, optimizer, num_epochs= 50):
    l_train =[]
    l_val = []

    for epoch in range(num_epochs): 
        model.train()
        loss_tr=[]

        for it, (images_raw, images_out) in enumerate(train_loader):
            optimizer.zero_grad()           
            y, _ = model(images_raw)
            loss = loss_func(images_out, y)
            loss_tr.append(loss.cpu().data.item())
            loss.backward()
            optimizer.step()
       
       #Evaluation
        model.eval()
        loss_ts = []

        with torch.no_grad():
            for it, (images_raw, images_out) in enumerate(val_loader):
                y, _ = model(images_raw)
                loss = loss_func(images_out, y)
                loss_ts.append(loss.detach().cpu().numpy())

        print(f"Epoch {epoch}, training_loss: {np.mean(loss_tr)} , validation_loss {np.mean(loss_ts)}")

        l_train.append(np.mean(loss_tr))
        l_val.append(np.mean(loss_ts))
    
    return l_train, l_val


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_func = torch.nn.MSELoss()

In [ ]:
USE_PRETRAINED_MODEL=True

if USE_PRETRAINED_MODEL:
    model.load_state_dict(torch.load(f'Autoencoder/autoencoder_26_10_2023.pt'))
    print("Model loaded")

    display(Image('Autoencoder/Loss_autoencoder.png')) #this is the loss I have saved during the training of the autoencoder
    
else:
   l_train , l_val = train(model, train_loader, val_loader, loss_func, optimizer,  num_epochs=50) 
   #with 50 epochs it takes around 10 minutes (on the whole dataset) on my PC
   torch.save(model.state_dict(), 'Autoencoder/autoencoder.pt')
   print("Model saved")

   plt.plot(l_train, label="Train")
   plt.plot(l_val, label="Validation")
   plt.legend()
   plt.show()

In [ ]:
out, encoded = model(test_set.tensors[0])

In [ ]:
out = out.detach().numpy()
print(out.shape)

In [ ]:
idx=0

image = Xtest[idx].reshape((28,28))
image_out = out[idx].reshape((28,28))

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(image, cmap='gray')
ax[1].imshow(image_out, cmap='gray')
plt.show()

In [ ]:
encoded = encoded.detach().numpy()
print(encoded.shape)

In [ ]:
plt.figure(figsize=(10,8))
cmap = plt.get_cmap('jet', 10)
p = plt.scatter(encoded[:,0], encoded[:,1], c=ytest, cmap=cmap)
cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(labels_dict.values())
plt.show()

In [ ]:
#on the whole trainset
_, encoded = model(train_set.dataset.tensors[0])
encoded = encoded.detach().numpy()

plt.figure(figsize=(10,8))
cmap = plt.get_cmap('jet', 10)
p = plt.scatter(encoded[:,0], encoded[:,1], c=ytrain, cmap=cmap)
cb = plt.colorbar(p)
cb.ax.set_title('Class', fontsize=11)
cb.ax.set_yticklabels(labels_dict.values())
plt.show()
plt.close()

If you are interested in Variational Autoencoders, there is a lot of documentation (and examples) online. I suggest you use the ```pytorch``` framework other than ```tensorflow``` or ```keras```. 

Since most of you have not followed a course on Deep Learning yet, we refrain from the examples with VAE but be aware of their existance and usefullness.